In [1]:
import numpy as np
import math
# %timeit -n10 

### Fastest nth root

In [2]:
x = np.random.random((500, 500))
y = np.random.random((500, 500))

In [3]:
%timeit -n100 x**(1/40)

4.16 ms ± 99.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%timeit -n100 np.power(x, 1/40)

4.04 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Fastest random float [0, 1[

In [5]:
from random import random as fastRandom
%timeit -n40000 fastRandom()

45.5 ns ± 5.51 ns per loop (mean ± std. dev. of 7 runs, 40000 loops each)


In [6]:
%timeit -n40000 np.random.random()

330 ns ± 25.7 ns per loop (mean ± std. dev. of 7 runs, 40000 loops each)


### Fastest power

In [7]:
%timeit -n100 x ** y
%timeit -n100 np.power(x, y)

4.27 ms ± 72.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.03 ms ± 33 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit -n100 x ** 2
%timeit -n100 np.power(x, 2)

84.7 µs ± 7.15 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.15 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### ==> Numpy.power is faster for small powers
#### ==> Python ** is faster for large powers

### Fastest Sqrt

In [9]:
%timeit -n500 x ** (1/2)
%timeit -n500 np.sqrt(x)

197 µs ± 2.57 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
194 µs ± 8.51 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


### Fastest Descending sorting in numpy

In [10]:
a = np.random.rand(40,40,80)

In [11]:
%timeit -n100 np.flip(np.sort(a, axis=-1, kind='quicksort'), axis=-1)
%timeit -n100 np.sort(a, axis=-1, kind='quicksort')[::-1]
%timeit -n100 (-np.sort(-a, axis=-1, kind='quicksort'))

2.67 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.55 ms ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.65 ms ± 52.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Fastest derangment
i.e. random shuffling such that no element is at the same place as in the begining, i.e. without fixed point

In [12]:
def random_derangement_1(n):
    while True:
        v = np.arange(n)
        for j in np.arange(n - 1, -1, -1):
            p = np.random.randint(0, j+1)
            if v[p] == j:
                break
            else:
                v[j], v[p] = v[p], v[j]
        else:
            if v[0] != 0:
                return v

In [13]:
def random_derangement_2(N):
    original = np.arange(N)
    new = np.random.permutation(N)
    same = np.where(original == new)[0]
    while len(same) != 0:
        swap = same[np.random.permutation(len(same))]
        new[same] = new[swap]
        same = np.where(original == new)[0]
        if len(same) == 1:
            swap = np.random.randint(0, N)
            new[[same[0], swap]] = new[[swap, same[0]]]
    return new

In [14]:
import itertools

In [15]:
def random_derangement_3(n):
    list_ex = np.arange(n)
    i = filter(lambda p: not any(i1==i2 for i1,i2 in zip(list_ex, p)), itertools.permutations(list_ex, len(list_ex)))
    return next(i)

In [16]:
%timeit -n100 random_derangement_1(2000)

5.85 ms ± 455 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:
%timeit -n900 random_derangement_2(2000)

59.4 µs ± 4.13 µs per loop (mean ± std. dev. of 7 runs, 900 loops each)


In [18]:
# Wont run the test...
#%timeit -n100 random_derangement_3(2000)

In [37]:
%timeit -n300 np.random.permutation(2000)

35.8 µs ± 2.83 µs per loop (mean ± std. dev. of 7 runs, 300 loops each)


#### Result
random_derangement_2 is the clear winner

## Fastest random in multiple distribs

In [124]:
import scipy.stats as stats
# Middle | Ignores half for crossing over
middle = lambda n : int(n/2)
# Uniform | Okay but in nature crossing over isn't uniform
uniform = lambda n : np.random.randint(n)
# Uniform Middle | Better, but still ignores the extremis
uniformMiddle = lambda n: np.random.randint(int(n/2 - n/4), int(n/2 + n/4))
# Normal | Better, but small proba that x >= n or x < 0
normal = lambda n : int(np.random.normal(n*0.5, n*0.33))
# Normal truncated | Best solution, fits nature, but rather costly
normalTrucated = lambda n : int(stats.truncnorm(-1.5, 3-((3+n)/n), loc=n/2, scale=n*0.3).rvs(1)[0])
# Normal truncated | Best solution, fits nature, but rather costly
def normalTrucatedSingle(n):
    nrml = np.random.normal(n*0.5, n*0.33)
    if nrml < 0:
        return 0
    elif nrml >= n:
        return n-1
    return nrml
def normalTrucatedMultiple(n, size=1):
    return np.random.normal(n*0.5, n*0.33, size=size).astype(int).clip(0, n-1)

In [63]:
%timeit -n3000 middle(60000)

293 ns ± 21.1 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [64]:
%timeit -n3000 uniform(60000)

1.16 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [65]:
%timeit -n3000 uniformMiddle(60000)

1.73 µs ± 215 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [66]:
%timeit -n3000 normal(60000)

2 µs ± 939 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [73]:
%timeit -n300 normalTrucated(600)

590 µs ± 6.7 µs per loop (mean ± std. dev. of 7 runs, 300 loops each)


In [135]:
%timeit -n3000 normalTrucatedSingle(60000)

1.77 µs ± 466 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [134]:
%timeit -n3000 normalTrucatedMultiple(60000, 1)

3.8 µs ± 839 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [143]:
%timeit -n3000 normalTrucatedMultiple(60000, 500)

16.3 µs ± 922 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [144]:
%timeit -n3000 normalTrucatedMultiple(60000, 1500)

41.7 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [145]:
%timeit -n3000 normalTrucatedMultiple(60000, 5000)

129 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 3000 loops each)
